This notebook computes the integrated stellar flux at the top of the substellar column. This constant is then saved to file to be used in data preprocessing.

In [ ]:
import os
import h5py
import numpy as np

## Simulation Constants

In [21]:
nbin = 353
Radius_star = 695508000.0
Dist_planet_star = 104718509490

## Load Stellar Flux

In [56]:
ifile_star1 = "../ifile/stars.h5"

file_id = h5py.File(ifile_star1, 'r')

sunFlx = file_id['sunFlx'][:]
hatp7 = file_id['hatp7'][:]

StarF_h = sunFlx

file_id.close()

In [57]:
for l in range(nbin):
    if StarF_h[l] < 0:
        StarF_h[l] = 0.0

## Load Gas Absorption Variables

In [58]:
ifile_ktable = "../ifile/opacities_venus.h5"

file_id = h5py.File(ifile_ktable, 'r')

ktemp_h = file_id['TK'][:] # 1D array with the temperature values used to calculate the cross-sections
kpress_h = file_id['PK'][:] # 1D array with the pressure values used to calculate the cross-sections
kwave_h = file_id['wavmid'][:]
kwave_max_h = file_id['wavmax'][:]
kwave_min_h = file_id['wavmin'][:]
kopac_h = file_id['gasabs'][:] # 4D cube for the gas cross-sections: 353x20x20x20 = 2824000, where 353 is the number of spectral bins.
dg_h = file_id['DELG'][:] # 1D array with the weights for the gaussian integration and has a size of 20.

file_id.close()

In [59]:
print("Shape of ktemp_h: ", ktemp_h.shape)
print("Shape of kpress_h: ", kpress_h.shape)
print("Shape of kwave_h: ", kwave_h.shape)
print("Shape of kwave_max_h: ", kwave_max_h.shape)
print("Shape of kwave_min_h: ", kwave_min_h.shape)
print("Shape of kopac_h: ", kopac_h.shape)
print("Shape of dg_h: ", dg_h.shape)
print("Shape of StarF_h: ", StarF_h.shape)

Shape of ktemp_h:  (20,)
Shape of kpress_h:  (20,)
Shape of kwave_h:  (353,)
Shape of kwave_max_h:  (353,)
Shape of kwave_min_h:  (353,)
Shape of kopac_h:  (2824000,)
Shape of dg_h:  (20,)
Shape of StarF_h:  (353,)


In [60]:
kwave_int_h = np.zeros(shape=kwave_h.shape)
dkwave_h = np.zeros(shape=kwave_h.shape)

for i in range(nbin):
    kwave_int_h[i] = kwave_min_h[i]
    dkwave_h[i] = kwave_max_h[i] - kwave_min_h[i]

## Calculate Flux at the Top of the Model

In [61]:
coszi = 1
ftop_model = coszi * StarF_h * (Radius_star / Dist_planet_star)**2

In [62]:
dg = dg_h * np.dot(dkwave_h, ftop_model)

In [80]:
flux_scale_factor = np.sum(dg)
print(flux_scale_factor)

2791.486802476382


Save value of `beta` to file, such that it can be used in pre-processing.

In [81]:
# Save value of flux_scale_factor to flux_scale_factor.txt using full precision
np.savetxt('/home/ucaptp0/oasis-rt-surrogate/data/constants/flux_scale_factor.txt', [flux_scale_factor],fmt="%.12f")